In [62]:
%pip install -U --quiet torch
%pip install -U --quiet numpy
%pip install -U --quiet sentencepiece
%pip install -U --quiet transformers
%pip install -U --quiet bitsandbytes
%pip install -U --quiet accelerate
%pip install -U --quiet huggingface_hub
%pip install -U --quiet pinecone-client
%pip install -U --quiet Biopython
%pip install -U --quiet ollama
%pip install -U --quiet flask
%pip install -U --quiet fastapi
%pip install -U --quiet "uvicorn[standard]"

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.13.1+cu116 requires torch==1.12.1, but you have torch 2.2.0 which is incompatible.
scipy 1.9.2 requires numpy<1.26.0,>=1.18.5, but you have numpy 1.26.3 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [63]:
import torch
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer
from Bio import Medline
import ollama

In [64]:
if torch.cuda.is_available():
  device = "cuda:0"
else:
  device = "cpu"
device

'cuda:0'

In [65]:
records = {}
missed = 0

with open("pubmed_data") as stream:
    for article in Medline.parse(stream):

        if not "PMID" in article:
            missed += 1
            continue

        if not "TI" in article:
            missed += 1
            continue

        if not "AB" in article:
            missed += 1
            continue
        
        records[article["PMID"]] = article

In [66]:
model = SentenceTransformer("sentence-transformers/all-mpnet-base-v2")

In [67]:
index_name = 'pubmed'
pinecone = Pinecone(api_key="7377f10c-e728-4408-b2a6-3ac28daf468f")
index = pinecone.Index(index_name)

In [68]:
def retrieve_documents(question):
    results = index.query(vector=model.encode([question])[0].tolist(), top_k=5)
    return [res["id"] for res in results.matches]

In [69]:
def generate_prompt(question):

    context_ids = retrieve_documents(question)

    context = ""
    for id in context_ids:
        text = records[id]["AB"]
        context += f"""
DOCUMENT_ID: {id}
{text}
"""


    prompt = f"""ANSWER the following QUESTION soley based on the CONTEXT given. Use only CONTEXT relevant for the QUESTION. Keep your ANSWER as short and concise as possible. Mention the DOCUMENT-ID where appropriate. 

CONTEXT:
{context}

QUESTION:
{question}

ANSWER:
"""

    return prompt


In [72]:
from fastapi import FastAPI

app = FastAPI()


@app.get("/")
def read_root():
    return {"Hello": "World"}


@app.get("/generate/{question}")
def generate(question: str):
    prompt = generate_prompt(question)
    return {"answer": ollama.generate(model="llama2", prompt=prompt)["response"]}